# Recommender Systems Using GNNs

Credits: Machine Learning Alchemy

Playlist: https://www.youtube.com/watch?v=h1zxhx815Fk&list=PLcLdsfpLufYCJ_eg7VWuI7ROQT7SxUDGj&index=2

## Self-Supervised Learning for generating Graphs

### How do we construct the graph?
- We use **3.5** as the threshold for a positive rating.
- Any interaction above rating 3.5 is considered an **edge**

### What loss function do we use?
- Since this is self-supervised learning, we cannot rely on the rating labels to compute the loss function, hence we will **not** use RMSE
- We can use Bayesian Personalized Ranking (BPR) Loss -
    - A pairwise objective, which **encourages** the predictions of *positive* samples to be **higher** than the *negative* samples for each user 

$$L_{BPR} = -\sum_{u=1}^M\sum_{i \in N_u}\sum_{j \notin N_u} \ln \sigma(\hat y_{ui} - \hat y_{uj}) + \lambda \|E^{(0)}\|^2$$

where -
- $\hat y_{ui}$: Predicted score of a **positive** sample
- $\hat y_{uj}$: Predicted score of a **negative** sample
- $\lambda$: Regularizer parameter
- $E^{(0)}$: Node Feature Embedding/Matrix at Layer 0 for **all** the nodes

Finally, we aim to maximise $\hat y_{ui} - \hat y_{uj}$ (Score difference of positive and negative samples), hence the negative sign in the overall loss, as we will minimise the loss.

## Importing Packages

In [1]:
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy

import torch
from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul
import torch_geometric as pyg
from torch_geometric.utils import degree, structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric import nn as pyg_nn

## Load Dataset

Dataset link: https://grouplens.org/datasets/movielens/

In [2]:
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '../datasets/MovieLens/'), '../datasets/MovieLens/')

movie_path = '../datasets/MovieLens/ml-latest-small/movies.csv'
rating_path = '../datasets/MovieLens/ml-latest-small/ratings.csv'
user_path_path = '../datasets/MovieLens/ml-latest-small/users.csv'

Using existing file ml-latest-small.zip
Extracting ../datasets/MovieLens/ml-latest-small.zip


## EDA

In [3]:
rating_df = pd.read_csv(rating_path)
print(rating_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [4]:
print('Unique movies =',len(rating_df['movieId'].unique()))
print('Unique users =',len(rating_df['userId'].unique()))

Unique movies = 9724
Unique users = 610


In [5]:
rating_df.rating.value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [6]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


We can see that **user** and **movie ids'** max values are very large, while the total values are way lesser, hence preprocessing is required and we will use `sklearn.preprocessing.LabelEncoder()`, which will normalize the ids to the length.

## Pre-processing

In [7]:
lbl_user = preprocessing.LabelEncoder()
lbl_movie = preprocessing.LabelEncoder()

In [8]:
rating_df.userId = lbl_user.fit_transform(rating_df.userId.values)
rating_df.movieId = lbl_user.fit_transform(rating_df.movieId.values)

In [9]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,325.127564,3101.735561,3.501557,1.205946e+09
std,182.618491,2627.050983,1.042529,2.162610e+08
min,0.000000,0.000000,0.500000,8.281246e+08
25%,176.000000,900.000000,3.000000,1.019124e+09
50%,324.000000,2252.000000,3.500000,1.186087e+09
75%,476.000000,5095.250000,4.000000,1.435994e+09
max,609.000000,9723.000000,5.000000,1.537799e+09


## Load edges between users
Load edges between users and movies

In [10]:
def load_edges_from_csv(
        df, 
        src_index_col = 'userId', 
        dst_index_col = 'movieId',
        link_index_col = 'rating',
        rating_threshold = 3.5
    ):
    """
        Loads csv containing edges between users and items

        Arguments:
            src_index_col (str): column name of users
            dst_index_col (str): column name of items
            link_index_col (str): column name of user item interaction
            rating_threshold (int, optional): Threshold to determine positivity of 
            edge. Defaults to 3.5

        Returns:
            edge_index (list of list) --> (2 x N) matrix containing the node ids of
            N user-item edges N here is the number of interactions
    """
    
    edge_index = None

    # Constructing COO format edge_index from input rating events

    src = [user_id for user_id in df[src_index_col]]
    dst = [(movie_id) for movie_id in df[dst_index_col]]

    # Apply rating threshold
    # link_index_col: Rating column
    edge_attr = torch.from_numpy(df[link_index_col].values).view(-1, 1) >= rating_threshold
    # edge_attrs = torch.from_numpy(df[link_index_col].values).view(-1, 1)

    edge_index = [[],[]]
    for i in range(edge_attr.shape[0]): # Iterate the rows
        if edge_attr[i]: # Create edge between USER and MOVIE only if rating is >= threshold
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])
    return edge_index

In [11]:
edge_index = load_edges_from_csv(rating_df)
edge_index = torch.LongTensor(edge_index)
print(edge_index.shape)

torch.Size([2, 61716])


In [12]:
num_users = len(rating_df['userId'].unique())
num_movies = len(rating_df['movieId'].unique())
num_users, num_movies

(610, 9724)

In [13]:
num_interactions = edge_index.shape[1]
all_indices = [i for i in range(num_interactions)]
num_interactions

61716

Split the edges of the graph using a 80/10/10 train/validation/test split

In [14]:
train_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.2,
    random_state=1
)

val_indices, test_indices = train_test_split(
    test_indices,
    test_size=0.5,
    random_state=1
)

In [15]:
train_edge_index = edge_index[:, train_indices]
val_edge_index = edge_index[:, val_indices]
test_edge_index = edge_index[:, test_indices]
print(train_edge_index.shape, val_edge_index.shape, test_edge_index.shape, sep ='\n')

torch.Size([2, 49372])
torch.Size([2, 6172])
torch.Size([2, 6172])


In [16]:
print(torch.unique(train_edge_index[0]).size())
print(torch.unique(train_edge_index[1]).size())

torch.Size([608])
torch.Size([6682])


## Bipartite Graph Representation
A Graph which has two sets, where individual elements of one set ($U$) is only connected to an element of the other set ($V$) and never to itself. E.g. User nodes connected to movie nodes, but they won't be connected to each other, hence bipartite.

How do we get the adjacency matrix from a bipartite graph?
- We start from the interaction matrix $R$, where
    - Row index: Represents node $U$
    - Column index: Represents node $V$
- Here is how we convert an interaction matrix to an adjacency matrix $A$ -
$$A=\begin{pmatrix}
\mathbf{0} & \mathbf{R}\\
\mathbf{R^T} & \mathbf{0}
\end{pmatrix}$$

$$\mathbf{R} \in \mathbb{R}^{M \times N}$$

where $\mathbf{R}$ is the interaction matrix

### Sparse and Dense examples
Let us first understand how sparse functions work in **PyTorch**

In [17]:
dense = torch.randn(5, 5)
sparse = dense.to_sparse_coo()
sparse

tensor(indices=tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3,
                        3, 4, 4, 4, 4, 4],
                       [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
                        4, 0, 1, 2, 3, 4]]),
       values=tensor([ 0.0045,  0.6369,  1.3144, -0.4025, -0.3675,  1.8789,
                       0.6708,  1.3912, -0.8439, -0.6128,  1.2500, -0.5788,
                       0.3349, -0.2484,  0.5485, -2.8863, -0.1045,  0.0095,
                       0.0864, -0.9890, -0.9143,  0.0922, -0.0992, -0.8831,
                       1.8689]),
       size=(5, 5), nnz=25, layout=torch.sparse_coo)

In [18]:
sparse.indices().t()

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [1, 4],
        [2, 0],
        [2, 1],
        [2, 2],
        [2, 3],
        [2, 4],
        [3, 0],
        [3, 1],
        [3, 2],
        [3, 3],
        [3, 4],
        [4, 0],
        [4, 1],
        [4, 2],
        [4, 3],
        [4, 4]])

In [19]:
sparse.to_dense()

tensor([[ 0.0045,  0.6369,  1.3144, -0.4025, -0.3675],
        [ 1.8789,  0.6708,  1.3912, -0.8439, -0.6128],
        [ 1.2500, -0.5788,  0.3349, -0.2484,  0.5485],
        [-2.8863, -0.1045,  0.0095,  0.0864, -0.9890],
        [-0.9143,  0.0922, -0.0992, -0.8831,  1.8689]])

Using `SparseTensor` is more easy for calculating the adjacency matrix $A$

In [20]:
SparseTensor(
    row = sparse.indices()[0],
    col = sparse.indices()[1],
    sparse_sizes = sparse.size()
).to_dense()

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

When zeros, it saves the space by not storing them

In [21]:
dense = torch.zeros(5, 5)
sparse = dense.to_sparse_coo()
sparse

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(5, 5), nnz=0, layout=torch.sparse_coo)

### The conversion functions

In [22]:
def interaction2adjacency(input_edge_index, num_users = num_users, num_movies = num_movies):
    # input_edge_index.shape: (2, num_edges)
    R = torch.zeros((num_users, num_movies))
    for i in range(input_edge_index.shape[1]):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        R[row_idx][col_idx] = 1
    
    R_transpose = torch.transpose(R, 0, 1)
    adj_mat = torch.zeros((num_users+num_movies, num_users+num_movies))
    adj_mat[:num_users, num_users:] = R.clone() # Returns a copy of the tensor
    adj_mat[num_users:, :num_users] = R_transpose.clone()
    adj_mat_edge_index = adj_mat.to_sparse_coo()
    adj_mat_edge_index = adj_mat_edge_index.indices()
    return adj_mat_edge_index

In [23]:
def adjacency2interaction(input_edge_index, num_users = num_users, num_movies = num_movies):
    sparse_input_edge_index = SparseTensor(
        row = input_edge_index[0],
        col = input_edge_index[1],
        sparse_sizes = (num_users+num_movies, num_users+num_movies)
    )
    adj_mat = sparse_input_edge_index.to_dense()
    interact_mat = adj_mat[:num_users, num_users:]
    r_mat_edge_index = interact_mat.to_sparse_coo().indices()
    return r_mat_edge_index

In [24]:
train_edge_index = interaction2adjacency(train_edge_index)
val_edge_index = interaction2adjacency(val_edge_index)
test_edge_index = interaction2adjacency(test_edge_index)

In [25]:
print(train_edge_index)
print(train_edge_index.shape, end='\n\n')
# Doubled as the edges are now bi-direectional/undirected
print(val_edge_index)
print(val_edge_index.shape, end='\n\n')
print(test_edge_index)
print(test_edge_index.shape)

tensor([[    0,     0,     0,  ..., 10330, 10331, 10333],
        [  610,   612,   615,  ...,   183,   183,   330]])
torch.Size([2, 98744])

tensor([[    0,     0,     0,  ..., 10278, 10301, 10327],
        [  794,   811,  1120,  ...,   248,   330,   183]])
torch.Size([2, 12344])

tensor([[    0,     0,     0,  ..., 10301, 10324, 10332],
        [ 1161,  1399,  1465,  ...,   304,   183,   183]])
torch.Size([2, 12344])


### Helper Function to compute BPR Loss

`structured_negative_sampling` is a **PyG** library
- Samples a negative edge :obj: `(i,k)` for every positive edge
- :obj: `(i,j)` in the graph given by :attr: `edge_index`, and returns it as a tuple of the form :obj:`(i,j,k)`
- Example - 
    ```
    edge_index = torch.as_tensor([[0, 0, 1, 2],
                                [0, 1, 2, 3]])
    structured_negative_sampling(edge_index)
    Output: (tensor([0, 0, 1, 2]), tensor([0, 1, 2, 3]), tensor([2, 3, 0, 2]))
    ```

In [26]:
def sample_mini_batch(batch_size, edge_index):
    """
        Randomly samples indices of a minibatch given an adjacency matrix

        Args:
            batch_size (int): mini-batch size
            edge_index (torch.Tensor): (2 x N) list of edges

        Returns:
            tuple: (user indices, positive item indices, negative_item_indices)
    """
    edges = structured_negative_sampling(edge_index)
    # edges (tuple): (node1, node2, node3)
    # node1: size = num_edges
    # node2: size = num_edges
    # node3: size = num_edges
    # node1 --> node2: Is an actual edge (positive edge)
    # node1 --> node3: Is a negative sampled edge (Does not exist edge)

    edges = torch.stack(edges, dim = 0) # Tuple --> Tensor
    # New shape of edges (Tensor): (3 x num_edges)

    indices = random.choices([i for i in range(edges[0].shape[0])], k = batch_size)
    # Randomly samples 'batch_size' number of values from a list [0, 1, 2, .... num_edges]

    batch = edges[:, indices]
    # We obtain `batch_size` number of user to real (positive) edges and fake (negative) edges 
    return batch[0], batch[1], batch[2]

Example

In [27]:
user, pos, neg = sample_mini_batch(1, train_edge_index)
print(user, pos, neg)
print('User',user.item(),'--> Item',pos.item(),': Actual Edge')
print('User',user.item(),'--> Item',neg.item(),': Fake Edge')

tensor([490]) tensor([3042]) tensor([4660])
User 490 --> Item 3042 : Actual Edge
User 490 --> Item 4660 : Fake Edge


## Implementing LightGCN

### Light Graph Convolution
Between each layer, LightGCN uses the following propagation rule for user and item embeddings

$$e_u^{(k+1)} = \sum_{i \in N_u}\frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}}e_i^{(k)}$$

$$e_i^{(k+1)} = \sum_{u \in N_i}\frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}}e_u^{(k)}$$

where -
- $N_u$: Set of all neighbours of user $u$ (**Items** LIKED by **user** $u$)
- $N_i$: Set of all neighbours of item $i$ (**Users** who LIKED **item** $i$)
- $e_u^{(k)}$: $k^{th}$ layer user-embedding
- $e_i^{(k)}$: $k^{th}$ layer item-embedding

### Layer Combination and Model Prediction
The only trainable parameters of LightGCN are the 0th layer embeddings $e_u^{(0)}$ and $e_i^{(0)}$ for each user and item. We combine the embeddings obtained at each layer of propagation to form the final embeddings for all users and items, $e_u$ and $e_i$ via the following equation -

$$e_u=\sum_{k=0}^K \alpha_k e_u^{(k)}$$
$$e_i=\sum_{k=0}^K \alpha_k e_i^{(k)}$$

where -
- $\alpha_k$: Hyperparameter which weights the contribution of the $k^{th}$ layer embedding to the final embedding

The model prediction is obtained by taking the inner product of the final user and item embeddings

$$\hat y_{ui} = <e_u, e_i> = e_u^Te_i$$

### Matrix Form
In our implementation, we utilize the matrix form of LightGCN. We perform multi-scale diffusion to obtain the final embedding, which sums embeddings diffused across multi-hop scales. ($K$ layers)

$$E^{(K)}=\alpha_0E^{(0)} + \alpha_1\tilde AE^{(0)} + \alpha_2\tilde A^2E^{(0)} + ... + \alpha_K\tilde A^KE^{(0)}$$

where
- $E^{(0)} \in \mathbb{R}^{(M \times N)} \times T$: Stacked initial item and user embeddings where -
    - $M$: Number of users
    - $N$: Number of items
    - $T$: Dimension of each embedding
- $\tilde A = D^{-\frac{1}{2}}AD^{-\frac{1}{2}}$: Is a symmetrically normalized adjacency matrix, where $D$ is a degree matrix



The operation $D^{-\frac{1}{2}}AD^{-\frac{1}{2}}$ is performed by `torch_geometric.nn.conv.gcn_conv.gcn_norm()`

`gcn_norm()` returns a tuple - 

(original edge_index Tensor, Normalization values: Root of Degree Neighbours x Root of Degree of itself)

In [28]:
pyg_nn.conv.gcn_conv.gcn_norm(edge_index=train_edge_index,
                            add_self_loops=False)

(tensor([[    0,     0,     0,  ..., 10330, 10331, 10333],
         [  610,   612,   615,  ...,   183,   183,   330]]),
 tensor([0.0070, 0.0204, 0.0101,  ..., 0.1140, 0.1140, 0.1078]))

In [29]:
class LightGCN(pyg_nn.MessagePassing):
    def __init__(self,
                 num_users,
                 num_items,
                 embedding_dim = 64,
                 K = 3,
                 add_self_loops = False):
        """
            Initialises the LightGCN Model

            Args:
                num_users (int): Number of users
                num_items (int): Number of items
                embedding_dim (int, optional): Dimensionality of embeddings (Default = 64)
                K (int, optional): Number of message passing layers (Default = 3)
                add_self_loops (bool, optional): Whether to add self-loops for message passing (Default = False)
        """
        super(LightGCN, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.K = K
        self.add_self_loops = add_self_loops

        self.users_emb = nn.Embedding(num_embeddings = self.num_users, 
                                      embedding_dim = self.embedding_dim) # e_u^0
        self.items_emb = nn.Embedding(num_embeddings = self.num_items,
                                      embedding_dim = self.embedding_dim) # e_i^0
        
        # Filling the input tensor with values drawn from a normal distribution
        # According to the LightGCN paper, this performs better
        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index):
        """
            Forward pass of LightGCN model

            Args:
                edge_index (SparseTensor): Adjacency Matrix
            
            Returns:
                tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """
        # GCN Norm returns a tuple 
        # (original edge_index Tensor, Normalization values: Root of Degree Neighbours x Root of Degree of itself)
        edge_index_norm = pyg_nn.conv.gcn_conv.gcn_norm(edge_index=edge_index,
                                                        add_self_loops=self.add_self_loops)
        
        # Concat the user_embedding and item_embedding as the layer0 embedding matrix
        # Size will be (n_users + n_items) x embedding_dimension
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0
        
        # Save the layer0 embedding to the embs list
        embs = [emb_0]

        # emb_k is the embedding that we are actually going to push it through the graph layers
        # as described the LightGCN paper formula
        emb_k = emb_0

        # Push the embedding of all users and items through the Graph Model K times
        # K here is the number of layers
        # This performs the "Matrix Form" formula mentioned before
        for i in range(self.K):
            # `propagate()` is a function from `MessagePassing` superclass
            # Calls the message() function when we call `propagate()`
            emb_k = self.propagate(edge_index=edge_index_norm[0],
                                   x = emb_k,
                                   norm = edge_index_norm[1])
            embs.append(emb_k)
        
        # Stacked embs is a LIST of embedding matrices at each layer
        # Shape: num_nodes x (n_layers + 1) x embedding_dim
        # Converts the list to Tensor format
        embs = torch.stack(embs, dim = 1)

        # In experiments, setting alpha_k = 1/(K+1) gives best results, so we take mean
        emb_final = torch.mean(embs, dim = 1) # E^K

        # Splits to e_u^k and e_i^k
        users_emb_final, items_emb_final = torch.split(emb_final, [self.num_users, self.num_items])

        return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight
    
    def message(self, x_j: torch.Tensor, norm) -> torch.Tensor:
        # When we call `propagate()`, we call message
        # x_j shape: edge_index_len x embedding length
        # x_j is the embedding of all the neighbours based on the src_list in coo_edge_index
        # element-wise multiply by the symmetical norm
        # Here we are using edge_index instead of adjacency matrix
        return norm.view(-1,1) * x_j

In [30]:
layers = 3
light_gcn = LightGCN(num_users=num_users,
                     num_items=num_movies,
                     K=layers)

# Loss Function



We utilize a Bayesian Personalized Ranking (BPR) loss, a pairwise objective which encourages the predictions of *positive* samples to be **higher** than *negative* samples for each user.

\begin{equation}
L_{BPR} = -\sum_{u = 1}^M \sum_{i \in N_u} \sum_{j \notin N_u} \ln{\sigma(\hat{y}_{ui} - \hat{y}_{uj})} + \lambda ||E^{(0)}||^2 
\end{equation}

$\hat{y}_{ui}$: predicted score of a positive sample

$\hat{y}_{uj}$: predicted score of a negative sample

$\lambda$: hyperparameter which controls the L2 regularization strength